In [1]:
!pip install ultralytics opencv-python deep_sort_realtime --quiet

In [2]:
from ultralytics import YOLO
import cv2
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np
import os
import time


In [3]:
model_path = 'model/best.pt'                      # Your YOLOv11 model
video_path = 'video/15sec_input_720p.mp4'         # Input football video
output_path = 'video/output_tracked.mp4'          # Output video path

os.makedirs('video', exist_ok=True)


In [4]:
model = YOLO(model_path)

from deep_sort_realtime.deepsort_tracker import DeepSort

tracker = DeepSort(
    max_age=60,
    n_init=2,
    max_iou_distance=0.7,
    max_cosine_distance=0.3,   # Stricter match
    embedder="mobilenet",      # Use default appearance model
    half=True,
    bgr=True,
    embedder_gpu=False          # Set False if no GPU
)



In [5]:
cap = cv2.VideoCapture(video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))


In [6]:
print("✅ Available model classes:", model.names)

frame_count = 0
unique_ids = set()
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, verbose=False, conf=0.4)
    detections = []

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            class_name = model.names[cls].lower()

            if class_name == 'player':
                w = x2 - x1
                h = y2 - y1
                if w > 30 and h > 60:
                    detections.append(([x1, y1, w, h], conf, class_name))

    # Re-ID using standard DeepSORT update (no embed_only)
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        unique_ids.add(track_id)

        x1, y1, x2, y2 = map(int, track.to_ltrb())
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
        cv2.putText(frame, f'Player {track_id}', (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    out.write(frame)
    frame_count += 1
    print(f"Processed Frame: {frame_count}", end='\\r')

cap.release()
out.release()
cv2.destroyAllWindows()

end_time = time.time()

print(f"✅ Tracking complete!")
print(f"🎯 Total unique player IDs assigned: {len(unique_ids)}")
print(f"⏱️ Time taken: {end_time - start_time:.2f} seconds")
print(f"📦 Frames Processed: {frame_count}")
print(f"🧾 Unique IDs: {unique_ids}")


✅ Available model classes: {0: 'ball', 1: 'goalkeeper', 2: 'player', 3: 'referee'}
Processed Frame: 1\rProcessed Frame: 2\rProcessed Frame: 3\rProcessed Frame: 4\rProcessed Frame: 5\rProcessed Frame: 6\rProcessed Frame: 7\rProcessed Frame: 8\rProcessed Frame: 9\rProcessed Frame: 10\rProcessed Frame: 11\rProcessed Frame: 12\rProcessed Frame: 13\rProcessed Frame: 14\rProcessed Frame: 15\rProcessed Frame: 16\rProcessed Frame: 17\rProcessed Frame: 18\rProcessed Frame: 19\rProcessed Frame: 20\rProcessed Frame: 21\rProcessed Frame: 22\rProcessed Frame: 23\rProcessed Frame: 24\rProcessed Frame: 25\rProcessed Frame: 26\rProcessed Frame: 27\rProcessed Frame: 28\rProcessed Frame: 29\rProcessed Frame: 30\rProcessed Frame: 31\rProcessed Frame: 32\rProcessed Frame: 33\rProcessed Frame: 34\rProcessed Frame: 35\rProcessed Frame: 36\rProcessed Frame: 37\rProcessed Frame: 38\rProcessed Frame: 39\rProcessed Frame: 40\rProcessed Frame: 41\rProcessed Frame: 42\rProcessed Frame: 43\rProcessed Frame: 44\rPr